In [13]:
import json
import logging
from jinja2 import Template


def gen_tensorflow_client_string(generated_tensor_data, model_name):
    """
      Generate TensorFlow SDK in Python.
      Args:
        generated_tensor_data: Example is {"keys": [[1.0], [2.0]], "features": [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}
    """
    code_template = """#!/usr/bin/env 
    python import requests
    def main():
        endpoint = "http://127.0.0.1:8500"
        json_data = {"model_name": "{{ model_name }}", "data": {{ tensor_data }} }
        result = requests.post(endpoint, json=json_data)
        print(result.json())
    if __name__ == "__main__":
        main()
    """

    generated_tensor_data_string = json.dumps(generated_tensor_data)
    template = Template(code_template)
    generate_code = template.render(
        model_name=model_name, tensor_data=generated_tensor_data_string)
    logging.debug("Generate the code in Python:\n{}".format(generate_code))

    return generate_code


def gen_tensorflow_client(generated_tensor_data, model_name):
    generate_code = gen_tensorflow_client_string(generated_tensor_data, model_name)
    generated_code_filename = "client.py"
    with open(generated_code_filename, "w") as f:
        f.write(generate_code)
    logging.info('Save the generated code in {}, try "python {}"'.format(
        generated_code_filename, generated_code_filename))

In [110]:
from Arthur.api_server.client import Client
client = Client("http://127.0.0.1:9008/")
client.pred(n=10)

ApiflyError: Unable to connect to the server, please try again later.

In [106]:
from Arthur.api_server.client import Client
client = Client("http://127.0.0.1:9008/")
client.pred(n=10)

89

In [100]:
from Arthur.api_server.client import Client
client = Client("http://127.0.0.1:9009/")
client.pred(n=10)

KeyboardInterrupt: 

In [71]:
"""
:description:
    Object related storage
"""

import os
import sys
import abc
import shutil
import ftplib
import logging

class ObjectInterface(object):
    """
    object interface, abstract class. Should not be used directly
    """
    __metaclass__ = abc.ABCMeta
    def __init__(self, config):
        """
        :param config:
            dict like config, should contains at leat
            {
                'uri': 'xxxx',
                'user': 'xxxx',   # or stands for accesskey
                'passwords': 'xxxx', # or stands for secretkey
                'extra': some_object
            }
        """
        self._config = config

    def _validate_config(self, config, keys):
        """validate config if there's any missing items"""
        ret = True
        for key in keys:
            if not key in config:
                ret = False

        return ret

    @abc.abstractmethod
    def put(self, dest, localfile):
        """
        :param dest:
            system path
        :param localfile:
            localfile
        :return:
            {
                'returncode': 0 for success, others for failure,
                'msg': 'if any'
            }
        """

    @abc.abstractmethod
    def delete(self, path):
        """
        delete a file
        :param path:
            object path
        :return:
            {
                'returncode': 0 for success, others for failure,
                'msg': 'if any'
            }
        """

    @abc.abstractmethod
    def get(self, path, localpath):
        """
        get the object into localpath
        :return:
            {
                'returncode': 0 for success, others for failure,
                'msg': 'if any'
            }
        """

    @abc.abstractmethod
    def head(self, path):
        """
        get the object info
        :return:
           {
               'returncode': 0 for success, others for failure,
               'msg': 'if any'
               'objectinfo': {
                   'size': 1024, # at least have this one
                   'atime': 'xxxx.xx.xx', # optional
                   'mtime': 'xxxx.xx.xx', # optional
                   'ctime': 'xxxx.xx.xx', # optional
                   .......
               }
           }
        """

    @abc.abstractmethod
    def mkdir(self, path, recursive=True):
        """
        mkdir dir of a path
        :return:
            {
                'returncode': 0 for success, others for failure,
                'msg': 'if any'
            }
        """

    @abc.abstractmethod
    def rmdir(self, path, recursive=True):
        """rmdir of a path
        :return:
            {
                'returncode': 0 for success, others for failure,
                'msg': 'if any'
            }
        """
class LocalObjectSystem(ObjectInterface):
    """local object system"""

    def __init__(self, kvconfig=None):
        """
        initialize
        """
        config = {
            'uri': '//local/storage:',
            'user': None,   # or stands for accesskey
            'passwords': None, # or stands for secretkey
            'extra': None
        }
        ObjectInterface.__init__(self, config)

    def put(self, dest, localfile):
        """
        local object put == copy
        """
        ret = {
            'returncode': 0,
            'msg': 'success'
        }
        try:
            shutil.copy2(dest, localfile)
        # pylint: disable=W0703
        except Exception as error:
            ret['returncode'] = -1
            ret['msg'] = 'failed to put:{}'.format(error)
        return ret

    def delete(self, path):
        """delete a file in local"""
        ret = {
            'returncode': 0,
            'msg': 'success'
        }
        try:
            os.unlink(path)
        # pylint: disable=W0703
        except Exception as error:
            ret['returncode'] = -1
            ret['msg'] = 'failed to unlink file:{}, err:{}'.format(path, error)
        return ret

    def get(self, path, localpath):
        """
        get a file into localpath
        """
        return self.put(path, localpath)

    def head(self, path):
        """get the object info"""
        retcode = 0
        msg = 'ok'
        objectinfo = None
        if not os.path.exists(path):
            retcode = 255
            msg = 'file/dir not found'
        else:
            statinfo = os.stat(path)
            objectinfo =  {
                'size': statinfo.st_size,
                'atime': statinfo.st_atime,
                'mtime': statinfo.st_mtime,
                'ctime': statinfo.st_ctime
            }
        info_dict = {
            'returncode': retcode,
            'msg': msg,
            'objectinfo': objectinfo
        }
        return info_dict

    def mkdir(self, path, recursive=True):
        """
        mkdir
        """
        ret = {
            'returncode': 0,
            'msg': 'success'
        }
        func = os.makedirs
        if not recursive:
            func = os.mkdir
        try:
            func(path)
        except IOError as error:
            ret['returncode'] = -1
            ret['msg'] = 'failed to mkdir, err:{}'.format(error)
        return ret

    def rmdir(self, path, recursive=True):
        """
        rmdir
        """
        ret = {
            'returncode': 0,
            'msg': 'success'
        }
        func = os.rmdir
        if recursive:
            func = shutil.rmtree
        try:
            func(path)
        except IOError as error:
            ret['returncode'] = -1
            ret['msg'] = 'failed to rmdir, err:{}'.format(error)
        return ret

# vi:set tw=0 ts=4 sw=4 nowrap fdm=indent

In [72]:
store.put(y,'/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/data/')

{'msg': 'success', 'returncode': 0}

In [74]:
store.delete('/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/data/client.py.tar.gz')

{'msg': 'success', 'returncode': 0}

In [65]:
store = LocalObjectSystem()

In [14]:
print gen_tensorflow_client_string({'x':1}, 'model_name')

#!/usr/bin/env 
    python import requests
    def main():
        endpoint = "http://127.0.0.1:8500"
        json_data = {"model_name": "model_name", "data": {"x": 1} }
        result = requests.post(endpoint, json=json_data)
        print(result.json())
    if __name__ == "__main__":
        main()
    


In [9]:
gen_tensorflow_client({'x':1}, 'model_name')

In [ ]:
    def push_directory(self):
        with tempfile.TemporaryDirectory() as tmpdir:
            archive = self.archive(rootdir=".", output_dir=tmpdir)
            remote.put(archive, "/tmp/rorolite-project.tgz")

            with lcd(tmpdir):
                self.generate_supervisor_config(rootdir=tmpdir)

                supervisor_archive = self.archive(tmpdir, base_dir=".rorolite", filename="rorolite-supervisor")
                remote.put(supervisor_archive, "/tmp/rorolite-supervisor.tgz")

            with remote.cd(self.deploy_root):
                remote.sudo("chown {} .".format(env.user))
                remote.run("tar xzf /tmp/rorolite-project.tgz")
                remote.run("tar xzf /tmp/rorolite-supervisor.tgz")

In [ ]:
APIFLY_FUNCTIONS="APIFLY_FUNCTIONS"
APIFLY_TOKEN="APIFLY_TOKEN"
def _run_server(apifly_function, apifly_token, host, port, workers=2, is_reg=False,static_prefix=None,
                gunicorn_opts=None):
    """
    Run the Arthur api server, wrapping it in gunicorn
    :param static_prefix: If set, the index.html asset will be served from the path static_prefix.
                          If left None, the index.html asset will be served from the root path.
    :return: None
    """
    env_map = {}
    if apifly_function:
        env_map[APIFLY_FUNCTIONS] = apifly_function
    if apifly_token:
        env_map[APIFLY_TOKEN] = apifly_token
    if static_prefix:
        env_map[STATIC_PREFIX_ENV_VAR] = static_prefix

    bind_address = "%s:%s" % (host, port)
    bind_address2 = "http//%s:%s" % (host, port)
    if is_reg:
        redis_db=api.data_api(redis_db=3).redis_api(use_type='mem',prefix='REST:SERVICE_LIST:')
        reg_info={}
        funcs_info=load_functions(apifly_function.split(","))
        addr=bind_address2
        token=apifly_token
        reg_info['addr']=addr
        reg_info['token']=token
        reg_info['funcs_list']=funcs_info
        redis_db['REST_SERVICE_REG']=json.dumps(reg_info)
    opts = shlex.split(gunicorn_opts) if gunicorn_opts else []
    exec_cmd(["gunicorn"] + opts + ["-b", bind_address, "-w", "%s" % workers,
                                    "--worker-class","gevent",
                                    "Arthur.api_server.main:app"],env=env_map, stream_output=True)

In [ ]:
ARTHUR_SERVICE_CONFIG = """
[program:{name}]
command = {command}
directory = {directory}
redirect_stderr = true
stdout_logfile = /var/log/arthur_service/%(program_name)s.log
environment =
    PATH="/opt/arthur_service/project/.rorolite/env/bin:%(ENV_PATH)s"
"""
_run_server(self.func, self.token, self.host, self.port)
class deploy(object):
    def _generate_config(self, service, rootdir):
        print("generating supervisor config for " + service['name'])
        name = service['name']
        function = service.get('function')
        command = service.get('command')
        port = service['port']
        directory = "/opt/arthur_service/project/" + service.get("directory", "")

        if function:
            command = '/opt/arthur_service/project/.arthur_service/env/bin/arthur -b 0.0.0.0:{port} {function}'.format(port=port, function=function)

        if command is None:
            raise Exception("command is not specified for service {!r}".format(name))

        path = pathlib.Path(rootdir).joinpath(".rorolite", "supervisor", name + ".conf")
        if not path.parent.exists():
            path.parent.mkdir(parents=True)

        text = SUPERVISOR_CONFIG.format(name=name, directory=directory, command=command)
        with path.open("w") as f:
            f.write(text)

In [79]:
SUPERVISOR_CONFIG = """
[program:{name}]
command = {command}
directory = {directory}
redirect_stderr = true
stdout_logfile = /var/log/supervisor/%(program_name)s.log
environment =
    PATH="/opt/rorolite/project/.rorolite/env/bin:%(ENV_PATH)s"
"""
name='Arthur'
command = '/opt/rorolite/project/.rorolite/env/bin/firefly -b 0.0.0.0:{port} {function}'.format(port=6000, function='function')
text = SUPERVISOR_CONFIG.format(name=name, directory='./', command=command)
import pathlib
rootdir='./'
path = pathlib.Path(rootdir).joinpath(".Arthur", "supervisor", name + ".conf")
if not path.parent.exists():
    path.parent.mkdir(parents=True)
with path.open("w") as f:
    f.write(text.decode('utf-8'))

In [80]:
text

'\n[program:Arthur]\ncommand = /opt/rorolite/project/.rorolite/env/bin/firefly -b 0.0.0.0:6000 function\ndirectory = ./\nredirect_stderr = true\nstdout_logfile = /var/log/supervisor/%(program_name)s.log\nenvironment =\n    PATH="/opt/rorolite/project/.rorolite/env/bin:%(ENV_PATH)s"\n'

In [59]:
import shutil,os,sys
PY2 = (sys.version_info.major == 2)
if PY2:
    from backports import tempfile
else:
    import tempfile

class deploy(object):
    def deploy(self, async=False):
        print("Deploying project {}. This may take a few moments ...".format(self.name))
        with tempfile.TemporaryDirectory() as tmpdir:
            archive = self.archive(tmpdir)
            size = os.path.getsize(archive)
            store = LocalObjectSystem()
            store.put(archive,"/tmp/arthur-project.tgz")
            
            with open(archive, 'rb') as f:
                format = 'tar'
                response =  self.client.deploy(
                    project=self.name,
                    archived_project=f,
                    size=size,
                    format=format,
                    async=async
                )
            if async:
                return Task(response['task_id'], self.SERVER_URL)
            else:
                return response
    def archive(self, rootdir, output_dir=None, format='gztar', base_dir=".", filename='arthur-project'):
        output_dir = output_dir or rootdir
        base_name = os.path.join(output_dir, filename)
        return shutil.make_archive(base_name, format, root_dir=rootdir, base_dir=base_dir)
x=deploy()
y=x.archive('./',filename='client.py')

In [61]:
size = os.path.getsize(y)
size

163828

In [30]:
# 管道执行命令， 命令小输出还可以， 如果输出比较大推荐走文件输出流
def exec_cmd2(cmd):
    
    #Log.debug("cmd: %s" %(cmd))
    content = ""
    try:
        p = Popen(cmd, bufsize = 4096, stdout = PIPE, shell = True)
        
        while True:
            cont = p.stdout.read()
            if cont == "": break
            content += cont
            #Log.debug("contLen: %d" %(len(cont)))
            time.sleep(1)
        retState = p.wait()
        
        return retState, content
    except Exception, e:
        #Log.err("(%s)" %(traceback.format_exc()))
        return 255, "cmd(%s) err: %s" %(str(cmd), str(e))
from subprocess import * 
import time
cmd="ls /tmp/ 2>/dev/null || echo"
_,output=exec_cmd2(cmd)
versions = [int(v) for v in output.strip().split('\n') if v.isnumeric()]

AttributeError: 'str' object has no attribute 'isnumeric'

In [32]:
import fabric.api as remote
output = remote.run("ls /tmp/ 2>/dev/null || echo", quiet=True)

NetworkError: Name lookup failed for d

In [31]:
output.strip().split('\n')

['1',
 'com.apple.launchd.HDUlEZN3U1',
 'com.apple.launchd.mtVJBaiFwJ',
 'mongodb-27017.sock',
 'mysql.sock',
 'mysql.sock.lock',
 'powerlog',
 'signin_eventlist-CDP - Handle CDP enroll.plist']

In [25]:
output

(0,
 'com.apple.launchd.HDUlEZN3U1\ncom.apple.launchd.mtVJBaiFwJ\nmongodb-27017.sock\nmysql.sock\nmysql.sock.lock\npowerlog\nsignin_eventlist-CDP - Handle CDP enroll.plist\n')

In [23]:
class ShellCommandException(Exception):
    pass

import subprocess
def exec_cmd(cmd, throw_on_error=True, env=None, stream_output=False, cwd=None, cmd_stdin=None,
             **kwargs):
    """
    Runs a command as a child process.
    A convenience wrapper for running a command from a Python script.
    Keyword arguments:
    cmd -- the command to run, as a list of strings
    throw_on_error -- if true, raises an Exception if the exit code of the program is nonzero
    env -- additional environment variables to be defined when running the child process
    cwd -- working directory for child process
    stream_output -- if true, does not capture standard output and error; if false, captures these
      streams and returns them
    cmd_stdin -- if specified, passes the specified string as stdin to the child process.
    Note on the return value: If stream_output is true, then only the exit code is returned. If
    stream_output is false, then a tuple of the exit code, standard output and standard error is
    returned.
    """
    cmd_env = os.environ.copy()
    if env:
        cmd_env.update(env)

    if stream_output:
        child = subprocess.Popen(cmd, env=cmd_env, cwd=cwd, universal_newlines=True,
                                 stdin=subprocess.PIPE, **kwargs)
        child.communicate(cmd_stdin)
        exit_code = child.wait()
        if throw_on_error and exit_code is not 0:
            raise ShellCommandException("Non-zero exitcode: %s" % (exit_code))
        return exit_code
    else:
        child = subprocess.Popen(
            cmd, env=cmd_env, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE,
            cwd=cwd, universal_newlines=True, **kwargs)
        (stdout, stderr) = child.communicate(cmd_stdin)
        exit_code = child.wait()
        if throw_on_error and exit_code is not 0:
            raise ShellCommandException("Non-zero exit code: %s\n\nSTDOUT:\n%s\n\nSTDERR:%s" %
                                        (exit_code, stdout, stderr))
        return exit_code, stdout, stderr
cmd="ls /tmp/ 2>/dev/null || echo"
output=exec_cmd(cmd)

OSError: [Errno 2] No such file or directory

In [37]:
%%file get_filename.py
import sys,os
def get_filename(backstep=0):
    """
    Get the file name of the current code line.
    :param backstep:
        will go backward (one layer) from the current function call stack
    """
    return os.path.basename(
        sys._getframe(backstep + 1).f_code.co_filename)  # pylint:disable=W0212
print get_filename()

Overwriting get_filename.py


In [42]:
from cup import decorators
import cup
def is_port_used(port):
    """
    judge if the port is used or not (It's not 100% sure as next second, some
    other process may steal the port as soon after this function returns)
    :platform:
        linux only (netstat command used inside)
    :param port:
        expected port
    :return:
        return True if the port is used, False otherwise
    """
    #@decorators.needlinux
    def __is_port_used(port):
        """internal func"""
        cmd = "netstat -nl | grep ':%s '" % (port)
        ret = cup.shell.ShellExec().run(cmd, 10)
        if 0 != ret['returncode']:
            return False
        stdout = ret['stdout'].strip()
        if 0 == len(stdout):
            return False
        else:
            return True
    return __is_port_used(port)
is_port_used(9011)

False

In [48]:
import socket 
def get_host_ip(): 
    """
    get host ip address
    获取本机IP地址

    :return:
    """ 
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
    try: 
        s.connect(('8.8.8.8', 80)) 
        ip = s.getsockname()[0] 
    finally: 
        s.close() 
    return ip 
def is_port_used(ip, port): 
    """
    check whether the port is used by other program
    检测端口是否被占用

    :param ip:
    :param port:
    :return:
    """ 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    try: 
        s.connect((ip, port))
        return True
    except OSError: 
        return False 
    finally: 
        s.close()
    # 测试 
if __name__ == '__main__':
    host_ip = get_host_ip() 
    print(host_ip) 
    print(is_port_used(host_ip, 80))


192.168.100.245


error: [Errno 61] Connection refused

In [51]:
import random
def execshell_withpipe_ex(cmd, b_printcmd=True):
    """
    Deprecated. Recommand using ShellExec.
    """
    strfile = '/tmp/%s.%d.%d' % (
        'shell_env.py', int(os.getpid()), random.randint(100000, 999999)
    )
    os.mknod(strfile)
    cmd = cmd + ' 1>' + strfile + ' 2>/dev/null'
    os.system(cmd)
    if True == b_printcmd:
        print cmd
    fphandle = open(strfile, 'r')
    lines = fphandle.readlines()
    fphandle.close()
    os.unlink(strfile)
    return lines
def execshell_withpipe_str(cmd, b_printcmd=True):
    """
    Deprecated. Recommand using ShellExec.
    """
    return ''.join(execshell_withpipe_ex(cmd, b_printcmd))
def is_proc_alive(procname, is_whole_word=False, is_server_tag=False, filters=False):
    """
    Deprecated. Recommand using cup.oper.is_proc_exist
    """
    # print procName
    if is_whole_word:
        cmd = "ps -ef|grep -w '%s'$ |grep -v grep" % procname
    else:
        cmd = "ps -ef|grep -w '%s' |grep -v grep" % procname

    if is_server_tag:
        cmd += '|grep -vwE "vim |less |vi |tail |cat |more "'
    if filters:
        if isinstance(filters, str):
            cmd += "|grep -v '%s'" % filters
        elif isinstance(filters, list):
            for _, task in enumerate(filters):
                cmd += "|grep -v '%s'" % task
    cmd += '|wc -l'
    rev = execshell_withpipe_str(cmd, False)
    if int(rev) > 0:
        return True
    else:
        return False
is_proc_alive('web_main.py')

OSError: [Errno 1] Operation not permitted

In [119]:
def get_pid(process_path, grep_string):
    """
    will return immediately after find the pid which matches
    1. ps -ef|grep %s|grep -v grep|grep -vE "^[vim|less|vi|tail|cat|more] "
    '|awk '{print $2}'
    2. workdir is the same as ${process_path}
    :param process_path:
        process that runs on
    :param grep_string:
        ps -ef|grep ${grep_string}
    :return:
        return None if not found. Otherwise, return the pid
    """
    cmd = (
        'ps -ef|grep \'%s\'|grep -v grep|grep -vwE "vim |less |vi |tail |cat |more "'
        '|awk \'{print $2}\''
    ) % (grep_string)
    
    ret = cup.shell.ShellExec().run(cmd, 10)
   # print ret
    pids = ret['stdout'].strip().split('\n')
   # print pids
    if len(pids) == 0 or len(pids) == 1 and len(pids[0]) == 0:
        return None
    for pid in pids:
        for sel_path in ["cwd", "exe"]:
            cmd = 'ls -l /proc/%s/%s|awk \'{print $11}\' ' % (pid, sel_path)
            ret = cup.shell.ShellExec().run(cmd, 10)
            print ret
            pid_path = ret['stdout'].strip().strip()
            if pid_path.find(process_path) == 0:
                return pid
    return None

# end linux functionalities }}
# vi:set tw=0 ts=4 sw=4 nowrap fdm=indent
p='/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/src/web'
g='web_main'
print get_pid(p,g)

{'returncode': 0, 'stderr': 'ls: /proc/3610/cwd: No such file or directory\n', 'stdout': ''}
{'returncode': 0, 'stderr': 'ls: /proc/3610/exe: No such file or directory\n', 'stdout': ''}
None


In [118]:
p='/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/deployff'
g='5002'
print get_pid(p,g)

{'returncode': 0, 'stderr': '', 'stdout': '7963\n7968\n8437\n8438\n'}
['7963', '7968', '8437', '8438']
None


In [134]:
%%file shell_utils.py
# -- coding:utf-8 --
import os
import subprocess
import threading
import warnings
import signal

class ShellExec(object):  # pylint: disable=R0903
    """
    For shell command execution.
    ::
        from cup import shell
        shellexec = shell.ShellExec()
        # timeout=None will block the execution until it finishes
        shellexec.run('/bin/ls', timeout=None)
        # timeout>=0 will open non-blocking mode
        # The process will be killed if the cmd timeouts
        shellexec.run(cmd='/bin/ls', timeout=100)
    """

    def __init__(self):
        self._subpro = None
        self._subpro_data = None

    def __kill_process(self, pid):
        os.kill(pid, signal.SIGKILL)

    def kill_all_process(self, async_content):
        """
        to kill all process
        """
        for pid in async_content.child_list:
            self.__kill_process(pid)

    def get_async_run_status(self, async_content):
        """
        get the command's status
        """
        try:
            from cup.res import linux
            async_process = linux.Process(async_content.pid)
            res = async_process.get_process_status()
        except err.NoSuchProcess:
            res = "process is destructor"
        return res

    def get_async_run_res(self, async_content):
        """
        if the process is still running the res shoule be None,None,0
        """
        return async_content.ret

    def async_run(self, cmd, timeout):
        """
        async_run
        return a dict {uuid:pid}
        self.argcontent{cmd,timeout,ret,cmdthd,montor}
        timeout:returncode:999
        cmd is running returncode:-999
        """

        def _signal_handle():
            """
            signal setup
            """
            signal.signal(signal.SIGPIPE, signal.SIG_DFL)

        def _target(argcontent):
            argcontent.__subpro = subprocess.Popen(
                    argcontent.cmd, shell=True, stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    preexec_fn=_signal_handle)
            from cup.res import linux
            parent = linux.Process(argcontent.__subpro.pid)
            children = parent.children(True)
            ret_dict = []
            for process in children:
                ret_dict.append(process)
            argcontent.child_list = ret_dict

        def _monitor(start_time, argcontent):
            while(int(time.mktime(datetime.datetime.now().timetuple())) - int(start_time) <
                    int(argcontent.timeout)):
                time.sleep(1)
                if argcontent.__subpro.poll() is not None:
                    self._subpro_data = argcontent.__subpro.communicate()
                    argcontent.ret['returncode'] = argcontent.__subpro.returncode
                    argcontent.ret['stdout'] = self._subpro_data[0]
                    argcontent.ret['stderr'] = self._subpro_data[1]
                    return
            str_warn = (
                'Shell "%s"execution timout:%d. To kill it' % (argcontent.cmd,
                    argcontent.timeout)
            )
            argcontent.__subpro.terminate()
            argcontent.ret['returncode'] = 999
            argcontent.ret['stderr'] = str_warn

            for process in argcontent.child_list:
                self.__kill_process(process)
            del argcontent.child_list[:]

        argcontent = Asynccontent()
        argcontent.cmd = cmd
        argcontent.timeout = timeout
        argcontent.ret = {
            'stdout': None,
            'stderr': None,
            'returncode': -999
        }
        argcontent.__cmdthd = threading.Thread(target=_target, args=(argcontent,))
        argcontent.__cmdthd.start()
        start_time = int(time.mktime(datetime.datetime.now().timetuple()))
        argcontent.__cmdthd.join(0.1)
        argcontent.pid = argcontent.__subpro.pid
        argcontent.__monitorthd = threading.Thread(target=_monitor,
                args=(start_time, argcontent))
        argcontent.__monitorthd.start()
        #this join should be del if i can make if quicker in Process.children
        argcontent.__cmdthd.join(0.5)
        return argcontent

    def run(self, cmd, timeout):
        """
        refer to the class description
        :param timeout:
            If the cmd is not returned after [timeout] seconds, the cmd process
            will be killed. If timeout is None, will block there until the cmd
            execution returns
        :return:
            returncode == 0 means success, while 999 means timeout
            {
                'stdout' : 'Success',
                'stderr' : None,
                'returncode' : 0
            }
        E.g.
        ::
            import cup
            shelltool = cup.shell.ShellExec()
            print shelltool.run('/bin/ls', timeout=1)
        """

        def _signal_handle():
            """
            signal setup
            """
            signal.signal(signal.SIGPIPE, signal.SIG_DFL)

        def _target(cmd):
            self._subpro = subprocess.Popen(
                cmd, shell=True, stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                preexec_fn=_signal_handle
            )
            self._subpro_data = self._subpro.communicate()
        ret = {
            'stdout': None,
            'stderr': None,
            'returncode': 0
        }
        cmdthd = threading.Thread(target=_target, args=(cmd, ))
        cmdthd.start()
        cmdthd.join(timeout)
        if cmdthd.isAlive() is True:
            str_warn = (
                'Shell "%s"execution timout:%d. To kill it' % (cmd, timeout)
            )
            warnings.warn(str_warn, RuntimeWarning)
            self._subpro.terminate()
            ret['returncode'] = 999
            ret['stderr'] = str_warn
        else:
            self._subpro.wait()
            times = 0
            while self._subpro.returncode is None and times < 10:
                time.sleep(1)
                times += 1
            ret['returncode'] = self._subpro.returncode
            assert type(self._subpro_data) == tuple, \
                'self._subpro_data should be a tuple'
            ret['stdout'] = self._subpro_data[0]
            ret['stderr'] = self._subpro_data[1]
        return ret


def _do_execshell(cmd, b_printcmd=True, timeout=None):
    """
    do execshell
    """
    if timeout is not None and timeout < 0:
        raise cup.err.ShellException(
            'timeout should be None or >= 0'
        )
    if b_printcmd is True:
        print 'To exec cmd:%s' % cmd
    shellexec = ShellExec()
    return shellexec.run(cmd, timeout)


def execshell(cmd, b_printcmd=True, timeout=None):
    """
    执行shell命令，返回returncode
    """
    return _do_execshell(
        cmd, b_printcmd=b_printcmd, timeout=timeout)['returncode']


def kill9_byname(strname):
    """
    kill -9 process by name
    """
    fd_pid = os.popen("ps -ef | grep -v grep |grep %s \
            |awk '{print $2}'" % (strname))
    pids = fd_pid.read().strip().split('\n')
    fd_pid.close()
    for pid in pids:
        os.system("kill -9 %s" % (pid))
#kill9_byname('gunicorn')

Overwriting shell_utils.py


In [132]:
from shell_utils import ShellExec

In [133]:
def get_pid(process_path, grep_string):
    """
    will return immediately after find the pid which matches
    1. ps -ef|grep %s|grep -v grep|grep -vE "^[vim|less|vi|tail|cat|more] "
    '|awk '{print $2}'
    2. workdir is the same as ${process_path}
    :param process_path:
        process that runs on
    :param grep_string:
        ps -ef|grep ${grep_string}
    :return:
        return None if not found. Otherwise, return the pid
    """
    cmd = (
        'ps -ef|grep \'%s\'|grep -v grep|grep -vwE "vim |less |vi |tail |cat |more "'
        '|awk \'{print $2}\''
    ) % (grep_string)
    
    ret = ShellExec().run(cmd, 10)
   # print ret
    pids = ret['stdout'].strip().split('\n')
   # print pids
    if len(pids) == 0 or len(pids) == 1 and len(pids[0]) == 0:
        return None
    for pid in pids:
        for sel_path in ["cwd", "exe"]:
            cmd = 'ls -l /proc/%s/%s|awk \'{print $11}\' ' % (pid, sel_path)
            ret = cup.shell.ShellExec().run(cmd, 10)
            print ret
            pid_path = ret['stdout'].strip().strip()
            if pid_path.find(process_path) == 0:
                return pid
    return pids

# end linux functionalities }}
# vi:set tw=0 ts=4 sw=4 nowrap fdm=indent
p='/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/src/web'
g='web_main'
print get_pid(p,g)

Exception in thread Thread-74:
Traceback (most recent call last):
  File "/Users/leepand/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/leepand/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "shell_utils.py", line 149, in _target
    self._subpro = subprocess.Popen(
NameError: global name 'subprocess' is not defined



AttributeError: 'NoneType' object has no attribute 'wait'

In [136]:
# -*- coding: utf-8 -*- 
import socket 
def get_host_ip(): 
    """
    get host ip address
    获取本机IP地址

    :return:
    """ 
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
    try: 
        s.connect(('8.8.8.8', 80)) 
        ip = s.getsockname()[0] 
    finally: 
        s.close() 
    return ip 
print get_host_ip()

192.168.100.245


In [137]:
#!/usr/bin/python
# -*- coding:utf8 -*-

import urllib2
import re

url = urllib2.urlopen("http://txt.go.sohu.com/ip/soip")
text = url.read()
ip = re.findall(r'\d+.\d+.\d+.\d+',text)

print ip[0]

1.203.175.22
